In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
againstEmpty = pd.read_csv('playAgainstEmptyNet.csv')
withEmpty = pd.read_csv('playWithEmptyNet.csv')
teamStats = pd.read_csv('teamStats.csv')
pullStats = pd.read_csv('pullStats.csv')

In [ ]:
teamStats['Team'] = withEmpty['Team']
teamStats['Team'] = teamStats['Team'].fillna('LEAGUE')
teamStats = teamStats.drop(['AvAge', ], axis=1)
teamStats = teamStats[["Team", "GP", "GF", "GA","PIM/G","oPIM/G", "PP", "PPA"]]

In [18]:
def optimalPullTime(teamA,teamB):

    def evenSR(a,b):
        AGoalsPerGame = (teamStats.iloc[a]["GF"]- teamStats.iloc[a]["PP"])/teamStats.iloc[a]["GP"]
        BDefenseRatio = (teamStats.iloc[b]["GA"] - teamStats.iloc[b]["PPA"])/(teamStats.iloc[31]["GA"] - teamStats.iloc[31]["PPA"])
        print('Even Expected goals', (BDefenseRatio*AGoalsPerGame))
        print('Scoring Rate', (BDefenseRatio*AGoalsPerGame)/(60 - teamStats.iloc[a]["PIM/G"] - teamStats.iloc[a]["oPIM/G"]))
        return (BDefenseRatio*AGoalsPerGame)/(60 - teamStats.iloc[a]["PIM/G"] - teamStats.iloc[a]["oPIM/G"])

    def APullSR(a,b):
        AGoalsPerGame = withEmpty.iloc[a]["Scored"]/withEmpty.iloc[a]["Count"]
        BDefenseRatio = (againstEmpty.iloc[b]["Allowed"]/againstEmpty.iloc[b]["Count"])/(againstEmpty["Allowed"]/againstEmpty["Count"]).mean()
        if BDefenseRatio == 0:
            BDefenseRatio = (0.1/againstEmpty.iloc[b]["Count"])/(againstEmpty["Allowed"]/againstEmpty["Count"]).mean()
        print("Expected # of Goals", BDefenseRatio*AGoalsPerGame)
        print("Scoring Rate",BDefenseRatio*AGoalsPerGame/pullStats.iloc[a]["avgPull"])
        return BDefenseRatio*AGoalsPerGame/pullStats.iloc[a]["avgPull"]

    def BPullSR(b,a):
        BGoalsPerGame = againstEmpty.iloc[b]["Scored"]/againstEmpty.iloc[b]["Count"]
        ADefenseRatio = (withEmpty.iloc[a]["Allowed"]/withEmpty.iloc[a]["Count"])/(againstEmpty["Allowed"]/againstEmpty["Count"]).mean()
        if ADefenseRatio == 0:
            ADefenseRatio = (0.1/againstEmpty.iloc[b]["Count"])/(againstEmpty["Allowed"]/againstEmpty["Count"]).mean()
        print("Expected # of Goals", ADefenseRatio*BGoalsPerGame)
        print("Scoring Rate",ADefenseRatio*BGoalsPerGame/pullStats["avgPull"].mean())
        return ADefenseRatio*BGoalsPerGame/pullStats["avgPull"].mean()

    def calculatePullTime(aE, bE, aP, bP):    
        return (-np.log((-aP*bE + aE*bP) / (aP*bP + aP**2 - aP*aE - aP*bE))) / (aP + bP)
    
    print("5v5")
    print("Team A")
    aSR = evenSR(teamA,teamB)
    print("Team B")
    bSR = evenSR(teamB,teamA)

    print("\n6v5")
    print("Team A")
    aPullSR = APullSR(teamA,teamB)
    print("Team B")
    bPullSR = BPullSR(teamB,teamA)

    return calculatePullTime(aSR, bSR, aPullSR, bPullSR)

In [19]:
# 0 : "ANAHEIM DUCKS"
# 1 : "ARIZONA COYOTES" 
# 2 : "BOSTON BRUINS"
# 3 : "BUFFALO SABRES"
# 4 : "CALGARY FLAMES"
# 5 : "CAROLINA HURRICANES"
# 6 : "CHICAGO BLACKHAWKS"
# 7 : "COLORADO AVALANCHE"
# 8 : "COLUMBUS BLUE JACKETS"
# 9 : "DALLAS STARS"
# 10 : "DETROIT RED WINGS"
# 11 : "EDMONTON OILERS"
# 12 : "FLORIDA PANTHERS"
# 13 : "LOS ANGELES KINGS"
# 14 : "MINNESOTA WILD"
# 15 : "MONTREAL CANADIENS"
# 16 : "NASHVILLE PREDATORS"
# 17 : "NEW JERSEY DEVILS"
# 18 : "NEW YORK ISLANDERS"
# 19 : "NEW YORK RANGERS"
# 20 : "OTTAWA SENATORS"
# 21 : "PHILADELPHIA FLYERS"
# 22 : "PITTSBURGH PENGUINS"
# 23 : "SAN JOSE SHARKS"
# 24 : "ST. LOUIS BLUES"
# 25 : "TAMPA BAY LIGHTNING"
# 26 : "TORONTO MAPLE LEAFS"
# 27 : "VANCOUVER CANUCKS"
# 28 : "VEGAS GOLDEN KNIGHTS"
# 29 : "WASHINGTON CAPITALS"
# 30: "WINNIPEG JETS"

TeamA = 26
TeamB = 15

print("\nTeam ", TeamA," VS Team ",  TeamB, "\nOptimal Pull Time: ", optimalPullTime(TeamA, TeamB))

5v5
Team A
Even Expected goals 2.7252747252747254
Scoring Rate 0.05976479660690187
Team B
Even Expected goals 2.0732142857142857
Scoring Rate 0.04690530058177116

6v5
Team A
Expected # of Goals 0.3659256355010103
Scoring Rate 0.2152503738241237
Team B
Expected # of Goals 0.5488884532515155
Scoring Rate 0.3512464061256944

Team  26  VS Team  15 
Optimal Pull Time:  3.895026702326681


In [ ]:
# THIS IS THE BLOCK THAT CALCULATED THE AVERAGE OPTIMAL PULL TIME

# leaguePullTimes = pd.DataFrame(columns=["matchUp", "pullTime"])

# for i in range(31):
#     for j in range(31):
#         if(i!=j):
#             matchup = pd.DataFrame([[str(i) + "-" + str(j) , optimalPullTime(i,j)]], columns=["matchUp", "pullTime"])
#             leaguePullTimes = pd.concat([leaguePullTimes, matchup], ignore_index = True)

# #setting the negatives to 0 to better contribute the league average
# leaguePullTimes[leaguePullTimes.pullTime <= 0] = 0

In [ ]:
# THIS IS WHERE THE AVERAGE AND STANDARD DEVIATION IS PRINTED
# print("Mean optimal Pull Time: ", leaguePullTimes["pullTime"].mean())
# print("Standard devation of average Pull Time: ", np.std(leaguePullTimes["pullTime"]))